In [ ]:
import sys, os
sys.path.insert(0,'/global/homes/b/bpb/metatlas/' )
sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
import pandas as pd
from metatlas import metatlas_objects as metob
from metatlas.helpers import dill2plots as dp
import numpy as np
import time
import copy
import dill
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 500)

In [ ]:
dp = reload(dp)
groups = dp.select_groups_for_analysis(name = '%nyogilipidsA%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['QC','Blank','extractioncontrol'])

# Functions

In [ ]:
def dataframe_container_from_metatlas_hdf5_file(my_file):
    data_df = pd.DataFrame()
    pd_h5_file  = pd.HDFStore(my_file)
    keys = pd_h5_file.keys()
    pd_h5_file.close()
    df_container = {}
    for k in keys:
        if ('ms' in k) and not ('_mz' in k):
            new_df = pd.read_hdf(my_file,k)
            df_container[k[1:]] = new_df
    return df_container


In [ ]:
import os
df = pd.DataFrame()
for group in groups:
    for my_file in group.items:
        container = dataframe_container_from_metatlas_hdf5_file(my_file.hdf5_file)
        for k in container.keys():
            if ('ms2' in k) and ('pos' in k):
                container[k]['polarity'] = 'positive'
                container[k]['hdf5_file'] = os.path.basename(my_file.hdf5_file)
                df = pd.concat([df,container[k]],axis=0)
            elif ('ms2' in k) and ('neg' in k):
                container[k]['polarity'] = 'negative'
                container[k]['hdf5_file'] = os.path.basename(my_file.hdf5_file)
                df = pd.concat([df,container[k]],axis=0)


In [ ]:
df.head()

In [ ]:
grouped = df.groupby(['precursor_MZ','rt','polarity','precursor_intensity','collision_energy','hdf5_file']).aggregate(lambda x: tuple(x))
grouped.mz = grouped.mz.apply(list)
grouped.i = grouped.i.apply(list)
grouped = grouped.reset_index()
grouped['spectrum'] = map(lambda x,y:(x,y),grouped['mz'],grouped['i'])
grouped['spectrum'] = grouped['spectrum'].apply(lambda x: zip(x[0],x[1]))
grouped.drop(['mz','i'], axis=1, inplace=True)


In [ ]:
print df.shape
print grouped.shape

In [ ]:
grouped.head()

In [ ]:
#Dump the mz rt table for mz mine filtering here

In [ ]:
target = 236.1485
min_intensity = 1e4
ppm = 10
def search_spectrum(spectrum,target,min_intensity,ppm):
    for s in spectrum:
        if (abs(s[0] - target) < target*ppm/1e6) & (s[1] > min_intensity):
            return True
    return False
grouped['has_target_fragment'] = grouped['spectrum'].apply(search_spectrum,args = (target,min_intensity,ppm))



In [ ]:
idx = (grouped.has_target_fragment) & (grouped.rt > 3.5) & (grouped.rt < 7.5) & (grouped.precursor_MZ > 500)
print len(idx)
df_sqdg = grouped[idx]

In [ ]:
df_sqdg = df_sqdg.sort_values('precursor_intensity',ascending=False)

In [ ]:
df_sqdg.head(10)

In [ ]:
df_unique_sqdg = pd.DataFrame()
df_unique_sqdg = df_sqdg.iloc[[0]]
for i in df_sqdg.index:
    idx = abs(df_sqdg.loc[i,'precursor_MZ'] - df_unique_sqdg.precursor_MZ) < 0.02
    if sum(idx) == 0:
        #The mz is new so add it
        df_unique_sqdg = pd.concat([df_unique_sqdg,df_sqdg.loc[[i]]],axis = 0)
    else:
        #the mz has been seen before; so see if the rt is new
        if sum(abs(df_unique_sqdg[idx].rt - df_sqdg.loc[i,'rt']) < 0.1)==0:
            df_unique_sqdg = pd.concat([df_unique_sqdg,df_sqdg.loc[[i]]],axis = 0)

In [ ]:
df_unique_sqdg.shape

In [ ]:
df_unique_sqdg.head(10)

In [ ]:
df_unique_sqdg.to_csv('/global/homes/b/bpb/Downloads/dgts_potential_hits.csv')

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
print(plt.style.available)

In [ ]:
for i in df_unique_sqdg.index:
    s = df_unique_sqdg.loc[i,'spectrum']
    mz = zip(*s)[0]
    intensity = zip(*s)[1]
    fig = plt.figure(figsize=(11,8.5))
    plt.style.use('seaborn-ticks')
    ax = plt.gca()
    ax.vlines(mz,intensity*0,intensity)
    ax.set_xlim(0,df_unique_sqdg.loc[i,'precursor_MZ']+5)
    ax.set_xlabel('m/z')
    ax.set_ylabel('intensity')
    sx = np.argsort(intensity)[::-1]
    labels = [1.001e9]
    for i_mz in sx:
        if np.min(np.abs(mz[i_mz] - labels)) > 0.1 and intensity[i_mz] > 0.02 * np.max(intensity):
            ax.annotate('%5.4f'%mz[i_mz], xy=(mz[i_mz], 1.01*intensity[i_mz]),rotation = 90, horizontalalignment = 'center', verticalalignment = 'left')
            labels.append(mz[i_mz])
    plt.tight_layout()
    L = plt.ylim()
    plt.ylim(L[0],L[1]*1.12)
    fig.savefig('/global/homes/b/bpb/Downloads/dgts_msms/msms_spectra_%d.png'%i)
    plt.close(fig)
